Трекинг эксперимента в ClearML

In [ ]:
%env CLEARML_WEB_HOST=https://app.clear.ml/
%env CLEARML_API_HOST=https://api.clear.ml
%env CLEARML_FILES_HOST=https://files.clear.ml
# My ML Project
%env CLEARML_API_ACCESS_KEY=
%env CLEARML_API_SECRET_KEY=

Импорт ClearML и библиотек

In [31]:
from clearml import Task
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import joblib

Инициализация эксперимента в ClearML

In [32]:
task = Task.init(
    project_name="Customer Return Prediction",
    task_name="Logistic Regression on train_sample",
    task_type="training"
)

ClearML Task: created new task id=490f7d7cf240400e890e5f4a5191946d
ClearML results page: https://app.clear.ml/projects/6ffd884e486546379d2150973a6c7e2e/experiments/490f7d7cf240400e890e5f4a5191946d/output/log


ClearML Monitor: GPU monitoring failed getting GPU reading, switching off GPU monitoring


Загрузка данных

In [ ]:
print("Загрузка train_sample.csv...")
train_sample = pd.read_csv('train_sample.csv')
print(f"Загружено: {len(train_sample):,} строк")

# Проверка
print("\nПервые 5 строк:")
print(train_sample.head())

Загрузка train_sample.csv...
Загружено: 42,746 строк

Первые 5 строк:
        client  Recency  Frequency  Monetary  total_quantity    avg_check  \
0      client1    587.0        1.0    2273.0             9.0  2273.000000   
1     client10     27.0        1.0    4757.0             3.0  4757.000000   
2    client100    116.0        1.0    7299.0             1.0  7299.000000   
3   client1000      8.0       12.0   31792.0           151.0  2649.333333   
4  client10000    396.0        1.0    8495.0             5.0  8495.000000   

   total_unique_items  avg_items_per_visit  weekend_visits  amount_last_visit  \
0                 4.0             9.000000             0.0              399.0   
1                 3.0             3.000000             0.0             4599.0   
2                 1.0             1.000000             0.0             7299.0   
3                28.0            12.583333            12.0             1359.0   
4                 5.0             5.000000             0.0    

Предобработка и обучение модели

In [34]:
# Удаляем client и выделяем y
X = train_sample.drop(columns=['client', 'event'])
y = train_sample['event'].astype(int)

# Только числовые признаки
X = X.select_dtypes(include='number')

print(f"X: {X.shape[1]} признаков, {X.shape[0]} объектов")
print(f"y: {len(y)} значений")

# Заполнение пропусков
imputer = SimpleImputer(strategy='median')
X_imputed = imputer.fit_transform(X)

# Масштабирование
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

print("Пропуски заполнены, признаки масштабированы")

X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)
print(f"X_train: {len(X_train):,} объектов")
print(f"X_test: {len(X_test):,} объектов")

print("Обучение модели...")
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(X_train, y_train)
print("Модель обучена")



X: 10 признаков, 42746 объектов
y: 42746 значений
Пропуски заполнены, признаки масштабированы
X_train: 34,196 объектов
X_test: 8,550 объектов
Обучение модели...
Модель обучена


Расчёт метрик и явное логирование

In [ ]:
# Предсказания и ROC-AUC
y_pred_proba = model.predict_proba(X_test)[:, 1]
roc_auc = roc_auc_score(y_test, y_pred_proba)
print(f"ROC-AUC: {roc_auc:.4f}")

# Логирование в ClearML
task.get_logger().report_scalar("Metrics", "ROC-AUC", roc_auc, iteration=1)

ROC-AUC: 0.7712


ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start
ClearML Monitor: Reporting detected, reverting back to iteration based reporting


Сохранение и логирование модели как артефакта

In [36]:
# Сохранение модели
joblib.dump(model, 'model.pkl')
joblib.dump(scaler, 'scaler.pkl') # от пропусков
joblib.dump(imputer, 'imputer.pkl') # нормализует признаки

# Прикрепление к ClearML
task.upload_artifact('model', 'model.pkl')

print("Модель и препроцессоры сохранены, модель прикреплена к таску в ClearML")


Модель и препроцессоры сохранены, модель прикреплена к таску в ClearML


Завершение эксперимента

In [37]:
task.close()
print("Эксперимент завершён!")

Эксперимент завершён!
